In [ ]:
!pip install tensorflow-gpu==2.0.0rc0
!pip install Pillow==6.2.0
!pip install scipy==1.3.0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, MaxPool2D, Reshape
from tensorflow.keras.layers import LSTM

In [ ]:
segments = np.load("<segments_file_name>.npy")
labels = np.load("<labels_file_name>.npy")

In [ ]:
tf.keras.backend.clear_session()

input = Input(shape=(2400,))
reshape_input = Reshape((1, 400, 6))(input)

x = Conv2D(32, kernel_size = (1, 24), 
        strides = (1, 1),  
        activation = "relu", 
        padding = "valid", 
        kernel_regularizer = l2(1e-4))(reshape_input)
x = MaxPool2D((1, 4), (1, 2))(x)

x = Conv2D(64, kernel_size = (1, 16), 
        strides = (1, 1), 
        activation = "relu", 
        padding = "valid", 
        kernel_regularizer = l2(1e-4))(x)
x = MaxPool2D((1, 4), (1, 2))(x)

x = Conv2D(96, kernel_size = (1, 8), 
        strides = (1, 1), 
        activation = "relu", 
        padding = "valid", 
        kernel_regularizer = l2(1e-4))(x)
x = MaxPool2D((1, 4), (1, 2))(x)

x = Conv2D(128, kernel_size = (1, 4), 
        strides = (1, 1), 
        activation = "relu", 
        padding = "valid", 
        kernel_regularizer = l2(1e-4),
        name="encoder")(x)        
x = Flatten()(x)
output = Dense(6, activation = "softmax")(x)

model = Model(input, output)
model.compile(optimizer = Adam(3e-4), 
              loss = "categorical_crossentropy",
              metrics = ["categorical_accuracy"])

In [ ]:
segments_reshaped = segments.reshape(-1, 2400)
model.fit(segments, labels, epochs = 1, 
          batch_size = 32, verbose = 2)

In [ ]:
!rm -R par_model
!mkdir par_model

model = Model(model.input, model.get_layer("encoder").output)
tf.keras.models.save_model(model, "par_model", 
                           include_optimizer = False, 
                           save_format="tf")

------------------

In [ ]:
from tfltransfer import bases
from tfltransfer import heads
from tfltransfer import optimizers
from tfltransfer.tflite_transfer_converter import TFLiteTransferConverter

In [ ]:
base = bases.SavedModelBase("par_model")

head = Sequential([
    # Flatten(input_shape=(1, 35, 128)),
    Dense(units=128,
        activation='relu',
        kernel_regularizer=l2(0.0001),
        input_shape=(256,)),
    Dense(units=2,
        activation='softmax',
        kernel_regularizer=l2(0.0001)),
])

# Optimizer is ignored by the converter! See docs.
head.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
!rm -R tflite_par_model
!mkdir tflite_par_model

converter = TFLiteTransferConverter(2,
              base,
              heads.KerasModelHead(head),
              optimizers.SGD(0.001),
              train_batch_size=5)

converter.convert_and_save('tflite_par_model')

--------------------